In [15]:
from heapq import heappop, heappush
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def get_goal_position(value: int) -> tuple[int, int]:
    if value == 0:
        return (PUZZLE_DIM - 1, PUZZLE_DIM - 1)
    value -= 1
    return (value // PUZZLE_DIM, value % PUZZLE_DIM)

def manhattan_distance(state: np.ndarray) -> int:
    dist = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:
                target_x, target_y = get_goal_position(value)
                dist += abs(x - target_x) + abs(y - target_y)
    return dist

def count_linear_conflicts(tiles: list[int], goals: list[int]) -> int:
    """Count linear conflicts in a row or column."""
    conflicts = 0
    for i in range(len(tiles)):
        if tiles[i] == 0: 
            continue
        for j in range(i + 1, len(tiles)):
            if tiles[j] == 0:  
                continue
            if goals[i] != -1 and goals[j] != -1:
                if goals[i] > goals[j]:
                    conflicts += 1
    return conflicts * 2  # Each conflict requires 2 additional moves

def count_inversions(state: np.ndarray) -> int:
    flat = [x for x in state.flatten() if x != 0]
    inversions = 0
    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inversions += 1
    return inversions

def is_solvable(state: np.ndarray) -> bool:
    n = state.shape[0]
    
    blank_row = [int(_[0]) for _ in np.where(state == 0)][0]
    blank_row_from_bottom = n - 1 - blank_row
    
    inversions = count_inversions(state)
    
    if n % 2 == 1:
        return inversions % 2 == 0
    else:
        return (inversions + blank_row_from_bottom) % 2 == blank_row_from_bottom % 2
        
def linear_conflict_distance(state: np.ndarray) -> int:
    """Calculate Manhattan distance plus linear conflicts."""
    base_distance = manhattan_distance(state)
    total_conflicts = 0
    
    for i in range(PUZZLE_DIM):
        row = state[i]
        goal_positions = []
        for tile in row:
            if tile != 0:
                goal_x, _ = get_goal_position(tile)
                goal_positions.append(goal_x if goal_x == i else -1)
            else:
                goal_positions.append(-1)
        total_conflicts += count_linear_conflicts(row.tolist(), goal_positions)
    
    for j in range(PUZZLE_DIM):
        column = state[:, j]
        goal_positions = []
        for tile in column:
            if tile != 0:
                _, goal_y = get_goal_position(tile)
                goal_positions.append(goal_y if goal_y == j else -1)
            else:
                goal_positions.append(-1)
        total_conflicts += count_linear_conflicts(column.tolist(), goal_positions)
    
    return base_distance + total_conflicts

def print_state(state: np.ndarray):
    print("\n".join([" ".join([str(cell).rjust(2) for cell in row]) for row in state]))

def a_star(initial_state: np.ndarray):
    if not is_solvable(initial_state):
        return None, 0, 0, 0, initial_state, None
        
    goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    open_set = []
    heappush(open_set, (0, initial_state.tobytes(), []))  
    state_mapping = {initial_state.tobytes(): initial_state}
    visited = set()
    visited.add(initial_state.tobytes())
    
    total_actions_evaluated = 0  

    while open_set:
        _, current_state_hash, path = heappop(open_set)
        current_state = state_mapping[current_state_hash]
        
        if np.array_equal(current_state, goal_state):
            quality = len(path)
            cost = total_actions_evaluated
            efficiency = quality / cost if cost > 0 else 0
            return path, quality, cost, efficiency, initial_state, goal_state

        for act in available_actions(current_state):
            total_actions_evaluated += 1
            new_state = do_action(current_state, act)
            state_key = new_state.tobytes()
            if state_key not in visited:
                visited.add(state_key)
                state_mapping[state_key] = new_state
                new_path = path + [act]
                cost = len(new_path) + linear_conflict_distance(new_state)
                heappush(open_set, (cost, state_key, new_path))

    return None, 0, 0, 0, initial_state, None

RANDOMIZE_STEPS = 10_000
while True:
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
        state = do_action(state, choice(available_actions(state)))
    
    if is_solvable(state):
        break
    print("Generated unsolvable state, trying again...")

print("\nInitial state:")
print_state(state)
print(f"Solvable: {is_solvable(state)}")

solution, quality, cost, efficiency, initial_state, goal_state = a_star(state)
if solution:
    print(f"\nSolution found in {quality} steps!")
    print(f"Quality: {quality}")
    print(f"Cost: {cost}")
    print(f"Efficiency: {efficiency:.4f}")
    
    print("\nGoal state (solution):")
    print_state(goal_state)
else:
    print("No solution found!")

Randomizing: 100%|██████████| 10000/10000 [00:00<00:00, 118599.42it/s]



Initial state:
 7  8  9 12
 3 13  5 15
 1  2  4 10
 6  0 11 14
Solvable: True

Solution found in 48 steps!
Quality: 48
Cost: 224749
Efficiency: 0.0002

Goal state (solution):
 1  2  3  4
 5  6  7  8
 9 10 11 12
13 14 15  0
